In [21]:
import random

import numpy as np
import torch
import yaml
from tqdm import tqdm

from utils import TryExcept
from utils.general import LOGGER, TQDM_BAR_FORMAT, colorstr

from sklearn.cluster import KMeans

In [22]:
dataset="./data/nuscenes.yaml"
n = 3

In [23]:
if isinstance(dataset, str):  # *.yaml file
    with open(dataset, errors="ignore") as f:
        data_dict = yaml.safe_load(f)  # model dict
    from utils.dataloaders import LoadImagesAndLabels

    dataset = LoadImagesAndLabels(data_dict["train"], augment=True, rect=True)

# Get label wh
shapes = img_size * dataset.shapes / dataset.shapes.max(1, keepdims=True)
wh0 = np.concatenate([l[:, 3:5] * s for s, l in zip(shapes, dataset.labels)])  # wh

# Filter
i = (wh0 < 3.0).any(1).sum()
wh = wh0[(wh0 >= 2.0).any(1)].astype(np.float32)  # filter > 2 pixels
# wh = wh * (npr.rand(wh.shape[0], 1) * 0.9 + 0.1)  # multiply by random scale 0-1

Scanning datasets/nuscenes/train.cache... 28130 images, 1425 backgrounds, 0 corrupt: 100%|██████████| 28130/28130 [00:00<?, ?it/s]


In [26]:
print(wh.shape[0])

268822


In [34]:
kmeans = KMeans(n_clusters=n)
kmeans.fit(wh)
anchors = kmeans.cluster_centers_

anchors = np.round(anchors).astype(int)
anchors = anchors[np.argsort(anchors[:, 0] * anchors[:, 1])]
anchors = anchors.reshape((3, 1, 2))

for i, scale in enumerate(["P3/8", "P4/16", "P5/32"]):
    print(f"Anchor for {scale}: {anchors[i][0]}")


Anchor for P3/8: [17 17]
Anchor for P4/16: [64 39]
Anchor for P5/32: [ 93 131]
